In [1]:
import pandas as pd
import random 
import json
import matplotlib.pyplot as plt

from functions import join_stocks_crypto, generate_rand_portfolios
from functions_post_clustering import reoptimize_weights, dunn_bonferroni

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
#GET THE DATA IN

#Main part data 2022-2023
df_all_stocks = pd.read_csv('stocks_data_FINAL.csv',index_col='Date')
df_all_stocks.index = pd.to_datetime(df_all_stocks.index)
df_all_stocks.index = df_all_stocks.index.strftime('%Y-%m-%d')

cryptos_df = pd.read_csv('cryptos_data_new.csv', index_col='timestamp')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')
joined_df.index = pd.to_datetime(joined_df.index)
returns_all = joined_df.pct_change().dropna()

tickers_to_select = list(joined_df.columns)
#Out-of-sample data 2024:
df_stocks_24 = pd.read_csv('stocks_data_out_sample_2024_FINAL.csv',index_col='Date')
cryptos_df_24 = pd.read_csv('cryptos_data_out_sample_2024.csv', index_col='timestamp')
joined_df_24 = join_stocks_crypto(cryptos_df_24, df_stocks_24, mode = 'stocks_left')
joined_df_24 = joined_df_24[tickers_to_select]
joined_df_24.index = pd.to_datetime(joined_df_24.index)
returns_all_24 = joined_df_24.pct_change().dropna()


tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

with open('equalw_sets.json') as f:
    equalw_sets = json.load(f)

with open('mdr_reoptimized_sets.json') as f:
    mdr_reoptimized_sets = json.load(f)



C:\Users\Kirill\AppData\Local\Temp\ipykernel_6648\2603468344.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_all_24 = joined_df_24.pct_change().dropna()


In [3]:
# def compare_crypto_overlap(portfolio1, portfolio2, crypto_assets):
#     """
#     Compare crypto overlap between two portfolios.
    
#     Parameters:
#     -----------
#     portfolio1, portfolio2 : list or dict
#         Portfolio assets (if dict, uses keys)
#     crypto_assets : list
#         List of crypto tickers to filter for
    
#     Returns:
#     --------
#     dict
#         Overlap statistics and details
#     """
#     # Extract assets from portfolios
#     if isinstance(portfolio1, dict):
#         assets1 = list(portfolio1.keys())
#     else:
#         print("AAAA")
#     #     assets1 = portfolio1
        
#     if isinstance(portfolio2, dict):
#         assets2 = list(portfolio2.keys())
#     else:
#         assets2 = portfolio2
    
#     # Filter for cryptos only
#     cryptos1 = set([asset for asset in assets1 if asset in crypto_assets])
#     cryptos2 = set([asset for asset in assets2 if asset in crypto_assets])
    
#     # Calculate overlap
#     intersection = cryptos1 & cryptos2
#     union = cryptos1 | cryptos2
    
#     # Calculate metrics
#     jaccard = len(intersection) / len(union) if len(union) > 0 else 0
#     overlap_pct = len(intersection) / min(len(cryptos1), len(cryptos2)) if min(len(cryptos1), len(cryptos2)) > 0 else 0
    
#     output =  {
#         'cryptos_portfolio1': sorted(list(cryptos1)),
#         'cryptos_portfolio2': sorted(list(cryptos2)),
#         'common_cryptos': sorted(list(intersection)),
#         'unique_to_p1': sorted(list(cryptos1 - cryptos2)),
#         'unique_to_p2': sorted(list(cryptos2 - cryptos1)),
#         'overlap_count': len(intersection),
#         'jaccard_similarity': jaccard,
#         'overlap_percentage': overlap_pct
#     }

#     if len(sorted(list(intersection))) < 3:
#         print('YES')
#     return 

# CRYPTO_ASSETS = list(cryptos_df.columns)

# for i in range(1000):
#     kmeans_port = crypto_supplemented_sets['kmeans_crypto'][f'portfolio_{i}']
#     kshape_port = crypto_supplemented_sets['kshape_crypto'][f'portfolio_{i}']

#     output = compare_crypto_overlap(kmeans_port, kshape_port, CRYPTO_ASSETS)


In [4]:
from functions_post_clustering import estimate_t_df_for_portfolio
import math

df_estimate = math.floor(estimate_t_df_for_portfolio(returns_all))
print(df_estimate)

Average df: 11.13, Min df: 5.71
11


SIMULATE AND EVALUATE

In [5]:
#FULL CLAUDE GENERATION CODE
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mstats, normaltest
import matplotlib.pyplot as plt
from datetime import timedelta
import random

# Missing function: calculate_cumulative_returns
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns from daily returns.
    
    Parameters:
    -----------
    returns : numpy.ndarray
        Array of daily returns
        
    Returns:
    --------
    numpy.ndarray
        Cumulative returns
    """
    return np.cumprod(1 + returns, axis=0) - 1

# Missing function: calculate_var_cvar
def calculate_var_cvar(returns, confidence_level=0.05, initial_value=100):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR)
    
    Parameters:
    -----------
    returns : numpy.ndarray
        Array of daily returns
    confidence_level : float
        Confidence level for VaR calculation (default: 0.05)
    initial_value : float
        Initial portfolio value (default: 100)
        
    Returns:
    --------
    tuple
        (VaR, CVaR)
    """
    VaR = abs(np.percentile(returns, confidence_level * 100)) * initial_value
    worst_losses = returns[returns <= -VaR/initial_value]
    CVaR = abs(worst_losses.mean()) * initial_value if len(worst_losses) > 0 else VaR
    
    return VaR, CVaR

def run_simulation(portfolio_dict, returns_for_portfolio, n_sims=100, t=100, 
                  distribution_model='multivar_norm', plot=False, initialPortfolio=100, 
                  winsorize=False, winsorize_limits=(0.01, 0.01), use_log_returns=False):
    """
    Run Monte Carlo simulation for portfolio returns.
    
    Parameters:
    -----------
    portfolio_dict : dict
        Dictionary with tickers as keys and weights as values
    returns_for_portfolio : pandas.DataFrame
        DataFrame with dates as index and tickers as columns containing returns
    n_sims : int
        Number of simulations to run
    t : int
        Number of time periods to simulate
    distribution_model : str
        Distribution model to use ('multivar_norm', 'multivar_t', 'bootstrap')
    plot : bool
        Whether to plot the simulation results
    initialPortfolio : float
        Initial portfolio value
    winsorize : bool
        Whether to winsorize returns
    winsorize_limits : tuple
        Limits for winsorization
    use_log_returns : bool
        Whether to use log returns (for consistency with out-of-sample analysis)
        
    Returns:
    --------
    numpy.ndarray
        Simulated portfolio paths
    """
    returns_for_portfolio = returns_for_portfolio[list(portfolio_dict.keys())]
    
    if winsorize:
        winsorized_returns = returns_for_portfolio.copy()
        
        for col in winsorized_returns.columns:
            winsorized_returns[col] = mstats.winsorize(winsorized_returns[col], limits=winsorize_limits)
        
        returns_for_portfolio = winsorized_returns

    # Convert to log returns if specified (for consistency with out-of-sample)
    if use_log_returns:
        invalid_mask = returns_for_portfolio <= -1.0
        if invalid_mask.any().any():
            log_compatible_returns = returns_for_portfolio.copy()
            log_compatible_returns[invalid_mask] = -0.99
            log_returns = np.log(1 + log_compatible_returns)
            returns_for_portfolio = log_returns
        else:
            returns_for_portfolio = np.log(1 + returns_for_portfolio)

    mean_returns = returns_for_portfolio.mean()
    cov_matrix = returns_for_portfolio.cov()

    weights = np.array([v for _, v in portfolio_dict.items()])

    meanM = np.tile(mean_returns, (t, 1))  # Shape: (T, n_assets)

    portfolio_sims = np.zeros((t, n_sims))

    L = np.linalg.cholesky(cov_matrix)  # Cholesky decomposition

    for sim in range(n_sims):
        if distribution_model == 'bootstrap':
            sampled_returns = returns_for_portfolio.bfill().sample(n=t, replace=True).values
            portfolio_returns = sampled_returns @ weights
        elif distribution_model in ['multivar_norm', 'multivar_t']:
            if distribution_model == 'multivar_norm':
                Z = np.random.normal(size=(t, len(portfolio_dict)))  # Shape: (T, n_assets)
                daily_returns = meanM + Z @ L.T  # Shape: (T, n_assets)
            elif distribution_model == 'multivar_t':
                df = 11  # degrees of freedom
                Z = np.random.normal(size=(t, len(portfolio_dict)))
                chi2 = np.random.chisquare(df, size=(t, 1))
                Z_t = Z / np.sqrt(chi2 / df)  # now Z_t has t-distributed marginals

                daily_returns = meanM + Z_t @ L.T

            portfolio_returns = daily_returns @ weights  # Shape: (T,)
        else:
            break
            
        # Convert back from log returns if necessary
        if use_log_returns:
            portfolio_returns = np.exp(portfolio_returns) - 1
        
        portfolio_sims[:, sim] = np.cumprod(1 + portfolio_returns) * initialPortfolio

    if plot:
        plt.figure(figsize=(10, 6))
        plt.plot(portfolio_sims)
        plt.title("Monte Carlo Simulated Portfolio Paths")
        plt.xlabel("Days")
        plt.ylabel("Portfolio Value")
        plt.show()

    return portfolio_sims

def out_of_sample_analysis(portfolio, return_data, start_date, period=126, 
                          calculate_metrics=True, plot_results=False, risk_free_rate=0.02, 
                          use_log_returns=False):
    """
    Perform traditional out-of-sample analysis for a portfolio over a specified forward period.
    
    Parameters:
    -----------
    portfolio : dict
        Dictionary with tickers as keys and weights as values
    return_data : pandas.DataFrame
        DataFrame with dates as index and tickers as columns containing RETURNS
    start_date : str or datetime
        Date marking the beginning of out-of-sample period
    period : int
        Number of days for the forward period to analyze (default 126 ~= 6 months of trading days)
    calculate_metrics : bool
        Whether to calculate and return performance metrics
    plot_results : bool
        Whether to generate and return performance plots
    risk_free_rate : float
        Annual risk-free rate used for Sharpe ratio calculation
    use_log_returns : bool
        Whether to use log returns (for consistency with simulation)
    
    Returns:
    --------
    results : dict
        Dictionary containing performance metrics and return series for the period
    """
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    
    results = {}
    tickers = list(portfolio.keys())
    weights = np.array([portfolio[ticker] for ticker in tickers])
    
    missing_tickers = [ticker for ticker in tickers if ticker not in return_data.columns]
    if missing_tickers:
        raise ValueError(f"The following tickers are missing from return_data: {missing_tickers}")
    
    return_subset = return_data[tickers].copy()
    return_subset = return_subset.sort_index()
    oos_data = return_subset[return_subset.index >= start_date]
    
    if oos_data.empty:
        raise ValueError(f"No data available after start_date: {start_date}")
    
    period_data = oos_data.iloc[:period]
    returns = period_data.copy()

    if use_log_returns:
        # Handle potential invalid values for log transformation
        invalid_mask = returns <= -1.0
        if invalid_mask.any().any():
            log_compatible_returns = returns.copy()
            log_compatible_returns[invalid_mask] = -0.99  # Cap extreme losses
            log_returns = np.log(1 + log_compatible_returns)
        else:
            log_returns = np.log(1 + returns)
            
        portfolio_log_returns = log_returns.dot(weights)
        
        # Convert back to simple returns for statistics
        portfolio_returns = np.exp(portfolio_log_returns) - 1
        
        # Calculate cumulative returns properly from log returns
        cumulative_returns = np.exp(portfolio_log_returns.cumsum()) - 1
    else:
        # Simple returns method
        portfolio_returns = returns.dot(weights)
        cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    results = {
        'returns': portfolio_returns,
        'cumulative_returns': cumulative_returns
    }
    
    if calculate_metrics:
        total_return = cumulative_returns.iloc[-1] if len(cumulative_returns) > 0 else 0
        trading_days = min(len(portfolio_returns), 252)
        
        # Annualization factor
        annual_factor = 252 / trading_days
        annualized_return = (1 + total_return) ** annual_factor - 1

        volatility = portfolio_returns.std() * np.sqrt(252)
        sharpe_ratio = (annualized_return - risk_free_rate) / volatility if volatility > 0 else 0
        
        # Non-annualized Sharpe ratio for the period
        rf_period = risk_free_rate * trading_days / 252
        volatility_period = portfolio_returns.std() * np.sqrt(trading_days)
        sharpe_period = (total_return - rf_period) / volatility_period if volatility_period > 0 else 0
        
        # Calculate VaR
        var_95 = abs(portfolio_returns.quantile(0.05)) * 100
        
        results.update({
            'mean_period_return': total_return,
            'annualised_return': annualized_return,
            'sharpe_annualized': sharpe_ratio,
            'sharpe_period': sharpe_period,
            'var_95': var_95,
        })
    
    return results


def calculate_diversification_ratio(portfolio_dict, return_df):
    """
    Calculate the diversification ratio for a portfolio.
    
    Diversification Ratio = (Weighted Average of Individual Asset Volatilities) / (Portfolio Volatility)
    
    A ratio > 1 indicates diversification benefits (portfolio volatility < weighted average of individual volatilities)
    A ratio = 1 indicates no diversification benefits (perfect correlation)
    A ratio < 1 is theoretically impossible for long-only portfolios
    
    Parameters:
    -----------
    portfolio_dict : dict
        Dictionary mapping ticker symbols to weights
    return_df : pandas.DataFrame
        DataFrame with returns data
        
    Returns:
    --------
    float
        Diversification ratio
    """
    try:
        # Get the assets in the portfolio
        assets = list(portfolio_dict.keys())
        weights = np.array([portfolio_dict[asset] for asset in assets])
        
        # Filter return data to only include portfolio assets
        portfolio_returns = return_df[assets].dropna()
        
        if len(portfolio_returns) < 30:  # Need minimum data for reliable volatility estimates
            return np.nan
        
        # Calculate individual asset volatilities (annualized)
        individual_volatilities = portfolio_returns.std() * np.sqrt(252)
        
        # Calculate weighted average of individual volatilities
        weighted_avg_volatility = np.sum(weights * individual_volatilities)
        
        # Calculate portfolio returns
        portfolio_return_series = (portfolio_returns * weights).sum(axis=1)
        
        # Calculate portfolio volatility (annualized)
        portfolio_volatility = portfolio_return_series.std() * np.sqrt(252)
        
        # Calculate diversification ratio
        if portfolio_volatility > 0:
            diversification_ratio = weighted_avg_volatility / portfolio_volatility
            return diversification_ratio
        else:
            return np.nan
            
    except Exception as e:
        print(f"Error calculating diversification ratio: {e}")
        return np.nan


def simulate_evaluate_portfolio_subset(portfolios_subset, return_df, n_sims=100, t=100, 
                                      distribution_model='multivar_norm', rf_annual=0.04, 
                                      seed=30, winsorize=False, winsorize_limits=(0.01, 0.01),
                                      use_log_returns=False):
    """
    Simulate and evaluate a subset of portfolios.
    
    Parameters:
    -----------
    portfolios_subset : dict
        Dictionary of dictionaries, where the outer keys are portfolio identifiers
        and the inner dictionaries are ticker->weight mappings
    return_df : pandas.DataFrame
        DataFrame with dates as index and tickers as columns containing RETURNS
    n_sims : int
        Number of simulations to run for each portfolio
    t : int
        Number of time periods to simulate
    distribution_model : str
        Distribution model to use ('multivar_norm', 'multivar_t', 'bootstrap', 'out_sample_direct')
    rf_annual : float
        Annual risk-free rate
    seed : int
        Random seed for reproducibility
    winsorize : bool
        Whether to winsorize returns
    winsorize_limits : tuple
        Limits for winsorization
    use_log_returns : bool
        Whether to use log returns for consistency between methods
        
    Returns:
    --------
    tuple
        (simulation_results, statistics, normality_test_results)
    """
    simulations_results_dict = dict()
    subset_statistics_df = pd.DataFrame()

    random.seed(seed)
    np.random.seed(seed)

    for i, portfolio_dict in portfolios_subset.items():
        # Making sure weights sum up to 1
        total = sum(portfolio_dict.values())
        portfolio_dict = {k: v / total for k, v in portfolio_dict.items()}
        
        # Calculate diversification ratio for this portfolio
        diversification_ratio = calculate_diversification_ratio(portfolio_dict, return_df)

        if distribution_model == 'out_sample_direct':
        
            res = out_of_sample_analysis(
                portfolio_dict, 
                return_df, 
                '2024-01-01', 
                period=t, 
                calculate_metrics=True, 
                risk_free_rate=rf_annual, 
                plot_results=False, 
                use_log_returns=use_log_returns
            )
            
            mean_annual_return = res['annualised_return']
            mean_return = res['mean_period_return']
            sharpe_annual = res['sharpe_annualized']
            VaR_final = res['var_95']
            sharpe_period = res['sharpe_period']

            stat_results = pd.DataFrame({
                'annualised_return': [mean_annual_return],
                'mean_period_return': [mean_return],
                'sharpe_annualized': [sharpe_annual],
                'sharpe_period': [sharpe_period],
                'VaR': [VaR_final],
                'diversification_ratio': [diversification_ratio],
            })
            
            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])
            
        else:
            # Run simulation with consistent parameters
            portfolio_sims = run_simulation(
                portfolio_dict, 
                return_df[list(portfolio_dict.keys())], 
                n_sims=n_sims, 
                t=t, 
                distribution_model=distribution_model, 
                plot=False, 
                winsorize=winsorize, 
                winsorize_limits=winsorize_limits,
                use_log_returns=use_log_returns
            )

            simulations_results_dict[i] = portfolio_sims
            
            daily_returns = (portfolio_sims[1:, :] - portfolio_sims[:-1, :]) / portfolio_sims[:-1, :]
            
            # For each simulation path, calculate key metrics
            all_stats = []
            
            for sim in range(n_sims):
                # Get the returns for this simulation
                sim_returns = daily_returns[:, sim]
                
                # Calculate cumulative return for the period
                final_value = portfolio_sims[-1, sim]
                initial_value = 100
                total_return = (final_value - initial_value) / initial_value
                
                # Calculate trading days (consistent with out-of-sample)
                trading_days = min(len(sim_returns), 252)
                
                # Annualization factor
                annual_factor = 252 / trading_days
                annualized_return = (1 + total_return) ** annual_factor - 1
                
                # Calculate volatility (annualized)
                volatility = sim_returns.std() * np.sqrt(252)
                
                # Calculate Sharpe ratio (annualized)
                sharpe_annual = (annualized_return - rf_annual) / volatility if volatility > 0 else 0
                
                # Calculate period Sharpe ratio
                rf_period = rf_annual * trading_days / 252
                volatility_period = sim_returns.std() * np.sqrt(trading_days)
                sharpe_period = (total_return - rf_period) / volatility_period if volatility_period > 0 else 0
                
                # Calculate VaR
                var_95 = abs(np.percentile(sim_returns, 5)) * 100
                
                all_stats.append({
                    'annualised_return': annualized_return,
                    'mean_period_return': total_return,
                    'sharpe_annualized': sharpe_annual,
                    'sharpe_period': sharpe_period,
                    'VaR': var_95
                })
            
            # Convert all simulations to a dataframe
            sim_stats_df = pd.DataFrame(all_stats)
            
            # Calculate averages across all simulations
            mean_annual_return = sim_stats_df['annualised_return'].mean()
            mean_return = sim_stats_df['mean_period_return'].mean()
            mean_sharpe_annual = sim_stats_df['sharpe_annualized'].mean()
            mean_sharpe_period = sim_stats_df['sharpe_period'].mean()
            mean_var = sim_stats_df['VaR'].mean()
            
            stat_results = pd.DataFrame({
                'annualised_return': [mean_annual_return],
                'mean_period_return': [mean_return],
                'sharpe_annualized': [mean_sharpe_annual],
                'sharpe_period': [mean_sharpe_period],
                'VaR': [mean_var],
                'diversification_ratio': [diversification_ratio],
            })

            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])

    subset_statistics_df = subset_statistics_df.reset_index(drop=True)

    # Run normality test (including diversification ratio)
    results_normality_test = {}
    for col in subset_statistics_df.columns:
        stat, p_value = normaltest(subset_statistics_df[col])
        results_normality_test[col] = {'statistic': stat, 'p_value': p_value}

    normality_results_df = pd.DataFrame(results_normality_test).T
    normality_results_df['normal'] = normality_results_df['p_value'] > 0.05

    return simulations_results_dict, subset_statistics_df, normality_results_df



In [ ]:
import os
folder_overall = 'NEW_RESULTS_FINAL/'
n_sims = 1000
data_returns = returns_all
data_name = 'simulation'
distribution_models = ['bootstrap', 'multivar_t']
time_periods = [126, 189, 252]
portfolio_sets_group = {'mdr_sets': mdr_reoptimized_sets, 'equalw_sets': equalw_sets}
windsorize = False
folder = 'simulation_results_no_windsor_log_returns_right_df_1000'


MODE = 'SIMULATION' 


if MODE == 'OUT_OF_SAMPLE_DIRECT':
    data_returns = returns_all_24
    data_name = 'out_sample_direct'
    distribution_models = ['out_sample_direct']
    windsorize = False
    folder = 'out_of_sample_direct_results_new_1000'

folder = folder_overall + folder
# Make sure output folders exist
os.makedirs(f'{folder}', exist_ok=True)

for set_name, set_itself in portfolio_sets_group.items():
    for distribution_model in distribution_models:
        for time_period in time_periods:
            subset_statistics_results_dfs = dict()
            normality_results_dfs = dict()
            results_all_df = pd.DataFrame()

            for key, portfolio_set in set_itself.items():
                simulations_results_dict, subset_statistics_df, normality = simulate_evaluate_portfolio_subset(
                    portfolio_set,
                    data_returns,
                    n_sims=n_sims,
                    t=time_period,
                    distribution_model=distribution_model,
                    winsorize=windsorize,
                    winsorize_limits=(0.01, 0.01),
                    use_log_returns=True
                )

                # Initialize nested dicts
                subset_statistics_results_dfs.setdefault(time_period, {})
                normality_results_dfs.setdefault(time_period, {})

                subset_statistics_results_dfs[time_period][key] = subset_statistics_df
                normality_results_dfs[time_period][key] = normality

                # Mean series and concat
                mean_series = subset_statistics_df.mean()
                mean_df = pd.DataFrame(mean_series, columns=[(time_period, key)])
                mean_df.columns = pd.MultiIndex.from_tuples(mean_df.columns)
                results_all_df = pd.concat([results_all_df, mean_df], axis=1)

            # Save to CSV with clear naming
            csv_filename = f"{folder}/stats_{set_name}_{distribution_model}_t{time_period}.csv"
            results_all_df.to_csv(csv_filename)

            # Run and save Dunn-Bonferroni tests
            all_dunn_results = dict()
            dunn_bonferroni_test_results = dunn_bonferroni(subset_statistics_results_dfs[time_period], metrics='all')
            all_dunn_results[time_period] = dunn_bonferroni_test_results

            # Save to Excel
            excel_filename = f"{folder}/dunn_matrix_{set_name}_{distribution_model}_t{time_period}.xlsx"
            with pd.ExcelWriter(excel_filename) as writer:
                for sheet_name, df in all_dunn_results[time_period].items():
                    df.to_excel(writer, sheet_name=sheet_name[:31])


            print(f'Done for {set_name}_{distribution_model}_t{time_period}, moving on')

Done for mdr_sets_bootstrap_t126, moving on
Done for mdr_sets_bootstrap_t189, moving on
Done for mdr_sets_bootstrap_t252, moving on
Done for mdr_sets_multivar_t_t126, moving on
Done for mdr_sets_multivar_t_t189, moving on
Done for mdr_sets_multivar_t_t252, moving on
Done for equalw_sets_bootstrap_t126, moving on
Done for equalw_sets_bootstrap_t189, moving on
Done for equalw_sets_bootstrap_t252, moving on
Done for equalw_sets_multivar_t_t126, moving on
Done for equalw_sets_multivar_t_t189, moving on
